Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## Langsmith Tracking and Tracing
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    productName: str = Field(description="The name of the product.")
    productDetails: str = Field(description="The details of the product.")
    productPrice: float = Field(description="The price of the product in USD.")



In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="qwen-qwq-32b")

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Product)

prompt_template = ChatPromptTemplate([
    ("system", "You are a product research assistant. Your task is to help the user find a product based on the product category provided. Response should contain product name in string, product details in string and product price in USD either float or interger."),
    ("user", "I need information about {productCategory} \n{format_instructions}")
])
format_instructions = parser.get_format_instructions()
prompt_template


ChatPromptTemplate(input_variables=['format_instructions', 'productCategory'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a product research assistant. Your task is to help the user find a product based on the product category provided. Response should contain product name in string, product details in string and product price in USD either float or interger.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'productCategory'], input_types={}, partial_variables={}, template='I need information about {productCategory} \n{format_instructions}'), additional_kwargs={})])

In [38]:
chain = prompt_template | model | parser
chain

ChatPromptTemplate(input_variables=['format_instructions', 'productCategory'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a product research assistant. Your task is to help the user find a product based on the product category provided. Response should contain product name in string, product details in string and product price in USD either float or interger.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'productCategory'], input_types={}, partial_variables={}, template='I need information about {productCategory} \n{format_instructions}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001685482B380>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000168549A81A0>, model_name='qwen-qwq-32b', model_kwargs={}

In [46]:
response = chain.invoke({"productCategory":"mobile phone",
                        "format_instructions": parser.get_format_instructions()
                        })
print(response)



{'productName': 'iPhone 15 Pro', 'productDetails': 'The iPhone 15 Pro features a dynamic island display, A17 Pro chip for advanced performance, 48MP main camera, titanium frame, and supports 5G. It includes an always-on Retina display and enhanced low-light photography capabilities.', 'productPrice': 999}
